<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 09:34:27
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -2.47 K (-0.02%)
Current PnL: -16.78 L (-11.29%)
CY Booked + Current PnL: -5.34 L (-3.59%)
-------------------
Total profit:  2.10 L
Total loss:  -18.88 L
-------------------
Total Booked + Current PnL: 21.64 L (17.78%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.29 L (56.92%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.00,62.0,H-MC,3.48,195170.0,30500.0,12823.0,-0.47,18.52,6.57,26.31,89.0,2.38,1.43,27.98,X40N,NTT,AC
76,TTKPRESTIG,770.00,103.84,55.0,M-SC,3.69,87803.0,-12974.0,13065.0,-0.50,-12.87,14.88,0.09,245.0,-0.99,0.64,15.35,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.42,62.0,H-MC,2.71,214875.0,23133.0,14633.0,0.44,12.06,6.81,19.69,99.0,1.58,1.57,19.68,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,111994.0,11490.0,17617.0,0.00,11.43,15.73,28.96,79.0,0.65,0.82,49.28,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219742.0,9376.0,19403.0,-0.45,4.46,8.83,13.68,37.0,0.48,1.61,21.72,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
67,STARHEALTH,761.0,11.22,59.0,H-SC,4.16,196972.0,-33947.0,132542.0,1.42,-14.70,67.29,42.70,144.0,-0.26,1.44,32.59,XY24,NTT,INSURANCE
21,COFFEEDAY,80.0,-41.65,52.0,L-SC,42.28,85248.0,-28301.0,68352.0,1.00,-24.92,80.18,35.27,268.0,-0.41,0.62,108.65,XR,NTT,HOTELS
44,KANSAINER,340.0,-66.69,67.0,H-SC,1.23,230184.0,-39483.0,75823.0,0.99,-14.64,32.94,13.47,138.0,-0.52,1.68,16.42,XY24,NTT,PAINTS
26,GREENPANEL,537.0,232.04,61.0,M-SC,1.84,148108.0,-34970.0,113954.0,0.95,-19.10,76.94,43.14,230.0,-0.31,1.08,39.32,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,-323.96,52.0,H-SC,6.26,124542.0,-39063.0,160373.0,-2.20,-23.88,128.77,74.15,139.0,-0.24,0.91,59.64,XR,NTT,CHEMICALS
53,RAJESHEXPO,518.0,1857.07,61.0,L-SC,2.66,53392.0,-83785.0,83879.0,-1.70,-61.08,157.10,0.07,267.0,-1.00,0.39,32.79,OX40N,NTT,JEWELLERY
52,QUESS,986.0,-40.50,48.0,M-SC,39.83,58988.0,-6018.0,156949.0,-1.19,-9.26,266.07,232.18,198.0,-0.04,0.43,2.73,XY24,NTT,MISC
4,ANGELONE,3033.0,4.08,37.0,H-SC,10.90,183869.0,-7137.0,64832.0,-1.03,-3.74,35.26,30.21,157.0,-0.11,1.35,16.36,X40N,NTT,FINANCE
50,NESTLEIND,1377.0,-8.86,54.0,H-LC,4.29,280285.0,14859.0,41931.0,-0.93,5.60,14.96,21.40,11.0,0.35,2.05,13.09,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,PGHH,17973.08,-30.94,51.0,H-MC,4.1,199875.0,-945.0,69716.0,-0.21,-0.47,34.88,34.25,80.0,-0.01,1.46,4.7,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,103.84,55.0,M-SC,3.69,87803.0,-12974.0,13065.0,-0.50,-12.87,14.88,0.09,245.0,-0.99,0.64,15.35,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.41,122850.0,-19629.0,39152.0,-0.02,-13.78,31.87,13.70,163.0,-0.50,0.90,45.00,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,67.0,H-SC,1.23,230184.0,-39483.0,75823.0,0.99,-14.64,32.94,13.47,138.0,-0.52,1.68,16.42,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,45.0,H-SC,2.10,145452.0,-30451.0,72479.0,-0.37,-17.31,49.83,23.89,149.0,-0.42,1.06,25.62,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.45,108424.0,-23411.0,65640.0,-0.47,-17.76,60.54,32.03,98.0,-0.36,0.79,21.44,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,46.75,68.0,M-MC,7.49,231265.0,6303.0,161354.0,0.84,2.80,69.77,74.53,192.0,0.04,1.69,35.96,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.00,62.0,H-MC,3.48,195170.0,30500.0,12823.0,-0.47,18.52,6.57,26.31,89.0,2.38,1.43,27.98,X40N,NTT,AC
83,VOLTAS,1530.00,2.42,62.0,H-MC,2.71,214875.0,23133.0,14633.0,0.44,12.06,6.81,19.69,99.0,1.58,1.57,19.68,XY25,NTT,AC
29,HAVELLS,2069.16,1.05,61.0,H-MC,1.94,250364.0,2633.0,72430.0,0.66,1.06,28.93,30.30,92.0,0.04,1.83,14.50,X40,ATH,ELECTRICAL
17,CAMS,5211.76,0.82,68.0,H-SC,3.45,114626.0,12622.0,31304.0,0.46,12.37,27.31,43.06,122.0,0.40,0.84,31.61,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,111994.0,11490.0,17617.0,0.00,11.43,15.73,28.96,79.0,0.65,0.82,49.28,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-41.04,49.0,M-SC,0.64,99630.0,8132.0,70618.0,-0.52,8.89,70.88,86.07,223.0,0.12,0.73,44.68,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.40,39.0,H-SC,11.52,133030.0,9694.0,124064.0,-0.52,7.86,93.26,108.45,119.0,0.08,0.97,31.97,AR,ATH,MISC
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158540.0,7595.0,101862.0,-0.47,5.03,64.25,72.51,53.0,0.07,1.16,11.36,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,65.0,H-MC,6.31,184268.0,4460.0,107944.0,0.67,2.48,58.58,62.52,88.0,0.04,1.35,37.41,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.43,41.0,H-LC,6.58,300945.0,-14897.0,127571.0,-0.39,-4.72,42.39,35.67,5.0,-0.12,2.20,8.48,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.74,218161.0,-33607.0,86828.0,0.02,-13.35,39.80,21.13,27.0,-0.39,1.60,17.35,X40,ATH,PAINTS
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200979.0,841.0,20058.0,-0.13,0.42,9.98,10.44,4.0,0.04,1.47,6.01,X40,NTT,FMCG
30,HCLTECH,1943.91,1.94,52.0,H-LC,9.70,232611.0,-9313.0,72575.0,-0.82,-3.85,31.20,26.15,8.0,-0.13,1.70,9.92,X40,ATH,IT
79,UNITDSPR,1644.00,-12.80,53.0,H-LC,7.76,228743.0,-5401.0,54029.0,0.19,-2.31,23.62,20.77,86.0,-0.10,1.67,4.45,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-0.29,67.0,H-LC,1.46,247072.0,921.0,78297.0,-0.62,0.37,31.69,32.18,16.0,0.01,1.81,36.55,X200,ATH,IT
41,ITC,452.0,-37.92,51.0,H-LC,2.37,200979.0,841.0,20058.0,-0.13,0.42,9.98,10.44,4.0,0.04,1.47,6.01,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.86,54.0,H-LC,4.29,280285.0,14859.0,41931.0,-0.93,5.60,14.96,21.40,11.0,0.35,2.05,13.09,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,56.0,H-LC,4.95,219742.0,9376.0,19403.0,-0.45,4.46,8.83,13.68,37.0,0.48,1.61,21.72,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.74,215090.0,10590.0,30822.0,-0.52,5.18,14.33,20.25,10.0,0.34,1.57,14.36,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.80,53.0,H-LC,7.76,228743.0,-5401.0,54029.0,0.19,-2.31,23.62,20.77,86.0,-0.10,1.67,4.45,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.80,55.0,H-LC,12.30,296091.0,-49865.0,124714.0,-0.84,-14.41,42.12,21.64,1.0,-0.40,2.17,4.89,X40,ATH,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200979.0,841.0,20058.0,-0.13,0.42,9.98,10.44,4.0,0.04,1.47,6.01,X40,NTT,FMCG
82,VBL,671.64,-16.43,41.0,H-LC,6.58,300945.0,-14897.0,127571.0,-0.39,-4.72,42.39,35.67,5.0,-0.12,2.20,8.48,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.94,52.0,H-LC,9.70,232611.0,-9313.0,72575.0,-0.82,-3.85,31.20,26.15,8.0,-0.13,1.70,9.92,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.60,65.0,H-LC,26.98,166580.0,-19515.0,231913.0,0.42,-10.49,139.22,114.13,15.0,-0.08,1.22,22.67,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200979.0,841.0,20058.0,-0.13,0.42,9.98,10.44,4.0,0.04,1.47,6.01,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,54.0,H-LC,5.74,215090.0,10590.0,30822.0,-0.52,5.18,14.33,20.25,10.0,0.34,1.57,14.36,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.74,218161.0,-33607.0,86828.0,0.02,-13.35,39.80,21.13,27.0,-0.39,1.60,17.35,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,56.0,H-LC,4.95,219742.0,9376.0,19403.0,-0.45,4.46,8.83,13.68,37.0,0.48,1.61,21.72,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.80,55.0,H-LC,12.30,296091.0,-49865.0,124714.0,-0.84,-14.41,42.12,21.64,1.0,-0.40,2.17,4.89,X40,ATH,IT
39,INFY,2275.00,-16.37,56.0,H-LC,8.17,325890.0,13002.0,158676.0,-0.69,4.16,48.69,54.87,3.0,0.08,2.38,11.08,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200979.0,841.0,20058.0,-0.13,0.42,9.98,10.44,4.0,0.04,1.47,6.01,X40,NTT,FMCG
82,VBL,671.64,-16.43,41.0,H-LC,6.58,300945.0,-14897.0,127571.0,-0.39,-4.72,42.39,35.67,5.0,-0.12,2.20,8.48,X40N,ATH,FMCG
1,ABB,7934.00,-37.43,69.0,H-LC,9.84,261518.0,-101.0,119305.0,0.23,-0.04,45.62,45.57,7.0,-0.00,1.91,11.28,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7100.00,68.0,L-SC,5.23,78994.0,-14816.0,91436.0,1.84,-15.79,115.75,81.67,269.0,-0.16,0.58,52.98,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-41.65,52.0,L-SC,42.28,85248.0,-28301.0,68352.0,1.00,-24.92,80.18,35.27,268.0,-0.41,0.62,108.65,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-172.73,67.0,M-SC,3.25,84120.0,-18090.0,111476.0,0.73,-17.70,132.52,91.37,208.0,-0.16,0.62,36.55,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-3.83,49.0,H-SC,12.51,89504.0,-11459.0,105991.0,0.30,-11.35,118.42,93.63,148.0,-0.11,0.65,33.31,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,51.0,H-SC,12.98,94230.0,-3750.0,25357.0,0.00,-3.83,26.91,22.05,152.0,-0.15,0.69,36.97,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1102.55,69.0,M-SC,4.57,163652.0,-17491.0,62957.0,0.35,-9.66,38.47,25.10,235.0,-0.28,1.20,31.15,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.0,3872.97,63.0,M-SC,5.34,123924.0,-23444.0,75259.0,-0.58,-15.91,60.73,35.16,236.0,-0.31,0.91,26.58,XY24,NTT,AUTO
74,TITAGARH,1548.0,1.18,69.0,H-SC,3.16,165900.0,-26115.0,104998.0,0.16,-13.60,63.29,41.08,158.0,-0.25,1.21,40.58,XY24,NTT,ENGINEERING
60,SAMMAANCAP,326.0,-172.73,67.0,M-SC,3.25,84120.0,-18090.0,111476.0,0.73,-17.70,132.52,91.37,208.0,-0.16,0.62,36.55,XY25,NTT,FINANCE
80,VAIBHAVGBL,521.0,64.74,70.0,H-SC,6.60,142861.0,-39914.0,150976.0,-0.55,-21.84,105.68,60.77,125.0,-0.26,1.05,27.96,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.07
1,25,43.90
2,50,74.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,44.55
LC,32.08
MC,23.35


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,29.38
X40,15.40
X40N,11.94
XR,11.80
AR,8.75
XY25,8.27
OX40N,8.19
X200,1.81
MH,1.70


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,29.30
H-LC,25.37
H-MC,20.35
M-SC,13.66
M-LC,5.64
M-MC,2.69
L-SC,1.59
L-LC,1.07
L-MC,0.31


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.61,-12.97,74.57
FMCG,11.89,-4.10,40.53
FINANCE,9.76,-15.38,60.57
MISC,7.47,-12.45,74.96
BANKS,6.33,-15.37,76.06
PAINTS,5.89,-14.70,31.80
ELECTRICAL,5.79,-4.02,61.35
AC,3.72,4.79,11.15
INSURANCE,3.47,-6.11,42.50


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2828116.0,22
AR,1299953.0,9
XR,1299312.0,14
X40,744375.0,10
X40N,573416.0,9
OX40N,525759.0,10
XY25,335679.0,6
SR,244001.0,2
X5K,119207.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3406552.0,29
M-SC,1380435.0,17
H-LC,1223795.0,14
H-MC,1086364.0,15
M-LC,379064.0,4
M-MC,310351.0,2
L-SC,243667.0,3
L-MC,60441.0,1
L-LC,38689.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1020126.0      6
M-SC       XY24       796334.0      7
H-SC       AR         787785.0      5
           XR         753679.0      7
H-MC       XY24       485637.0      4
H-LC       X40        462851.0      5
           AR         351218.0      2
M-MC       XY24       310351.0      2
H-SC       X40N       286660.0      4
           OX40N      250675.0      4
           SR         244001.0      2
H-LC       X40N       212422.0      3
H-MC       X40        211012.0      4
M-SC       AR         160950.0      2
L-SC       XR         159788.0      2
M-LC       XY24       157995.0      2
M-SC       OX40N      125565.0      4
H-MC       XY25       124180.0      2
M-LC       X5K        119207.0      1
M-SC       XR         115598.0      2
           XY25       111476.0      1
H-MC       XR         107944.0      1
M-LC       XR         101862.0      1
L-SC       OX40N       83879.0      1
H-LC       X200        78297.0      1
H-MC       X40N        74334.0      2
M-SC       X40         70512.0      1
H-MC       OX40N       65640.0      1
H-SC       MH          63626.0      1
H-LC       XY25        61334.0      2
L-MC       XR          60441.0      1
H-LC       XY24        57673.0      1
L-LC       XY25        38689.0      1
H-MC       MH          17617.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 89.0 seconds
